<a href="https://colab.research.google.com/github/aadegokemuideen/dino/blob/master/tfRegressionWithTitanicDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install necessary package sklean ,ipython and others

### This project explores Titanic data set:  The survival and non-survival people

In [ ]:
!pip install  -q sklearn
!pip install ipython

Use tensorflow 2.x  and import tensorflow as tf

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
from __future__ import absolute_import, division,print_function, unicode_literals

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc


## Let load Data. I am working on Titanic Data sets

In [ ]:
dftrain = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv") # training data
dfeval = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")   # eval/ testing data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [ ]:
dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [ ]:
dftrain.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [ ]:
dfeval.shape

(264, 9)

## categoricalColumn and numerical columns of dftrain are listed and  and converted to tf.feature_culumn.categorical_column_with_vocabulary_list  Format

In [ ]:
categoricalColumn = ['sex','class','deck','embark_town','alone' ]
numericColumn = ['age', 'fare']
feature_columns =[]

for c in categoricalColumn:
   vocabulary = dftrain[c].unique()
   feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(c,vocabulary))

for n in numericColumn:
  feature_columns.append(tf.feature_column.numeric_column(n,dtype=tf.float32))

In [ ]:
print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, default_value=-1, num_oov_buckets=0), NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# In this section, inner function name input_function() will be returned. ds is the random order of data, ds.batch(batch_size).repeat(num_epochs) will split the dataset into batches of 32 and repeat the process for number of given epochs. ds is the batch of the data set.

In [ ]:
def make_input_fn(data_df,label_df, num_epochs=10, shuffle=True, batch_size=32):
  def  input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df)) # This create data set object for us in tensorflow
    if shuffle:
      ds = ds.shuffle(1000) 
    ds = ds.batch(batch_size).repeat(num_epochs) # This repeat the process for given number of epochs.
    return ds
  return input_function

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs= 1, shuffle= False)

In this part linear estimator is creater and by pasing the feature columns 

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmprkspk_7e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Now the model created needs to be trained : To train a model a bit easy 
#train the model and evaluate it:(get the result in term of  dictioanry of stats about the model

In [ ]:
linear_est.train(train_input_fn) # train
result = linear_est.evaluate(eval_input_fn) #get model metrics/stats about the models using test data set

clear_output() # This clear output from the console
print(result["accuracy"])
print(result)

0.7613636
{'accuracy': 0.7613636, 'accuracy_baseline': 0.625, 'auc': 0.83027244, 'auc_precision_recall': 0.79283786, 'average_loss': 0.48287475, 'label/mean': 0.375, 'loss': 0.47772408, 'precision': 0.68367344, 'prediction/mean': 0.37796235, 'recall': 0.67676765, 'global_step': 400}


#predict

In [ ]:
result = list(linear_est.predict(eval_input_fn))
clear_output()
print(result[0])
print(result[0]['probabilities'][0])
# to check who sureve and  probability of some of surviavle or not surveive
print(dfeval.loc[4],'\n')
print(y_eval.loc[4], '\n')
print(result[4]['probabilities'][1])


{'logits': array([-2.6392267], dtype=float32), 'logistic': array([0.06665611], dtype=float32), 'probabilities': array([0.9333438 , 0.06665613], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
0.9333438
sex                          male
age                            34
n_siblings_spouses              0
parch                           0
fare                           13
class                      Second
deck                            D
embark_town           Southampton
alone                           y
Name: 4, dtype: object 

1 

0.27770364
